In [1]:
import os,sys
sys.path.append("/nashome/w/wketchum/analysis-CommonTools/")
from common_imports import *
from matplotlib_tools import *

In [2]:
import ROOT

Welcome to JupyROOT 6.22/08


In [3]:
ROOT.gInterpreter.AddIncludePath('$SBNOBJ_INC')
ROOT.gSystem.Load('$SBNOBJ_LIB/libsbnobj_Common_Calibration_dict.so')

0

In [4]:
%jsroot on

In [56]:
def prepare_df(df,selection):
    
    df = df.Filter(selection)
    
    df = df.Define("angle_azimuth","atan(dir.x/dir.z)")
    df = df.Define("angle_zenith","atan(sqrt(dir.x*dir.x+dir.y*dir.y)/dir.z)")
    
    for plane in range(3):
        hitstr="hits%d"%plane
        df = df.Define("{0}_n".format(hitstr),
                       "{0}.dir.x.size()".format(hitstr))
        df = df.Define("{0}_angle_azimuth".format(hitstr),
                       "atan({0}.dir.x.back()/{0}.dir.z.back())".format(hitstr))
        df = df.Define("{0}_angle_zenith".format(hitstr),
                       "atan(sqrt({0}.dir.x.back()*{0}.dir.x.back()+{0}.dir.y.back()*{0}.dir.y.back())/{0}.dir.z.back())".format(hitstr))
        for var in ["dqdx"]:
            df = df.Define("{0}_{1}_mean".format(hitstr,var),
                           "ROOT::VecOps::Mean({0}.{1})".format(hitstr,var))
        for hvar in ["integral","width"]:
            df = df.Define("{0}_{1}_mean".format(hitstr,hvar),
                           "ROOT::VecOps::Mean({0}.h.{1})".format(hitstr,hvar))
    
    return df

In [61]:
def make_histos_dict(df):
    hists = {}
    
    for c in ["x","y","z"]:
        hists["start_{0}".format(c)] = df.Histo1D("start.{0}".format(c))
        hists["end_{0}".format(c)] = df.Histo1D("end.{0}".format(c))
        hists["dir_{0}".format(c)] = df.Histo1D("dir.{0}".format(c))
    
    hists["length"] = df.Histo1D("length")
    hists["t0"] = df.Histo1D("t0")
    hists["angle_azimuth"] = df.Histo1D("angle_azimuth")
    hists["angle_zenith"] = df.Histo1D("angle_zenith")
    
    for p in range(3):
        for v in ["dqdx_mean","integral_mean","width_mean","angle_azimuth","angle_zenith","n"]:
            hists["hits{0}_{1}".format(p,v)] = df.Histo1D("hits{0}_{1}".format(p,v))
    
    return hists            

In [5]:
c = ROOT.TCanvas()

In [6]:
selection_stopping = "selected==0"

In [7]:
files_bnb_sim = ROOT.TFileCollection()
files_bnb_sim.Add("/pnfs/icarus/persistent/calibration/calib_ntuples/mc/ICARUS_BNB_Nu_Cosmics/hist_prodcorsika_bnb*.root")
print("Found %d BNB sim files in our list."%files_bnb_sim.GetNFiles())

Found 2291 BNB sim files in our list.


In [8]:
files_bnb_data = ROOT.TFileCollection()
files_bnb_data.Add("/pnfs/icarus/persistent/calibration/calib_ntuples/data/hist_data_dl4_fstrmBNB*.root")
print("Found %d BNB data files in our list."%files_bnb_data.GetNFiles())

Found 787 BNB data files in our list.


In [9]:
chain_bnb_sim = ROOT.TChain("caloskimE/TrackCaloSkim");
chain_bnb_sim.AddFileInfoList(files_bnb_sim.GetList())

1

In [65]:
chain_bnb_data = ROOT.TChain("caloskimE/TrackCaloSkim");
chain_bnb_data.AddFileInfoList(files_bnb_data.GetList())

1

In [66]:
df_bnb_sim_stopping = prepare_df(ROOT.RDataFrame(chain_bnb_sim),selection_stopping)
hists_bnb_sim_stopping = make_histos_dict(df_bnb_sim_stopping)

In [67]:
df_bnb_data_stopping = prepare_df(ROOT.RDataFrame(chain_bnb_data),selection_stopping)
hists_bnb_data_stopping = make_histos_dict(df_bnb_data_stopping)

In [37]:
df_check = df_bnb_sim_stopping

for plane in range(3):
    for angle in ["azimuth","zenith"]:
        df_check = df_check.Define("diff_{0}_hits{1}".format(angle,plane),
                                   "angle_{0}-hits{1}_angle_{0}".format(angle,plane))

In [68]:
hists_bnb_sim_stopping["hits2_dqdx_mean"].Draw()
c.Draw()

In [69]:
hists_bnb_data_stopping["hits2_dqdx_mean"].Draw()
c.Draw()

SegmentationViolation: TH1D& ROOT::RDF::RResultPtr<TH1D>::operator*() =>
    SegmentationViolation: segfault in C++; program state was reset

 *** Break *** segmentation violation
